In [270]:
import sys
if 'google.colab' in sys.modules:
    import os
    from google.colab import files
    # just check if we already uploaded, may we restart the runtime and run all cells
    if not os.path.isfile('DataErmeraTimorLeste.xlsx'):
        uploaded = files.upload()

In [271]:
import pandas as pd
import numpy as np
import pyomo.environ as pyo

In [272]:
data = pd.read_excel('mobian_data.xlsx', sheet_name=None, index_col=0)

In [273]:
demand = data['v'].reset_index()

distances_jh = data['d_jh'].reset_index()
distances_hp = data['d_hp'].reset_index()
distances_jp = data['d_jp'].reset_index()

time_jh = data['c_jh'].reset_index()
time_hp = data['b_hp'].reset_index()
time_jp = data['c_jp'].reset_index()

junctions = data['junctions']#.drop_duplicates()
hubs      = data['hubs']#.drop_duplicates()
pois      = data['pois']#.drop_duplicates()

junctions_index = set(junctions.index)
hubs_index      = set(hubs.index)
pois_index      = set(pois.index)

parameters = data['single_parameters'].reset_index().to_dict('records')[0]

In [274]:
demand.index   = list(junctions.index)
demand.columns = list(pois.index)
demand         = demand.T.reset_index().drop_duplicates().set_index('index').T
demand         = demand.reset_index().drop_duplicates().set_index('index')

distances_jh.index   = list(junctions.index)
distances_jh.columns = list(hubs.index)

distances_jh         = distances_jh.T.reset_index().drop_duplicates().set_index('index').T
distances_jh         = distances_jh.reset_index().drop_duplicates().set_index('index')

distances_hp.index   = list(hubs.index)
distances_hp.columns = list(pois.index)
distances_hp         = distances_hp.T.reset_index().drop_duplicates().set_index('index').T
distances_hp         = distances_hp.reset_index().drop_duplicates().set_index('index')

distances_jp.index   = list(junctions.index)
distances_jp.columns = list(pois.index)
distances_jp         = distances_jp.T.reset_index().drop_duplicates().set_index('index').T
distances_jp         = distances_jp.reset_index().drop_duplicates().set_index('index')

time_jh.index   = list(junctions.index)
time_jh.columns = list(hubs.index)
time_jh         = time_jh.T.reset_index().drop_duplicates().set_index('index').T
time_jh         = time_jh.reset_index().drop_duplicates().set_index('index')

time_hp.index   = list(hubs.index)
time_hp.columns = list(pois.index)
time_hp         = time_hp.T.reset_index().drop_duplicates().set_index('index').T
time_hp         = time_hp.reset_index().drop_duplicates().set_index('index')

time_jp.index   = list(junctions.index)
time_jp.columns = list(pois.index)
time_jp         = time_jp.T.reset_index().drop_duplicates().set_index('index').T
time_jp         = time_jp.reset_index().drop_duplicates().set_index('index')


In [275]:
v = data['d_jh'].reset_index()
v
#v.index = list(junctions.index)
#v.columns = list(hubs.index)

#v = v.reset_index().drop_duplicates().set_index('index')
#v.T.reset_index().drop_duplicates().set_index('index').T


,0,1,2,3,4,5,6,7,8,9,...,147,148,149,150,151,152,153,154,155,156
0,3.930,3.375,9.924,5.902,7.612,6.487,5.661,5.815,2.997,9.250,...,4.138,4.255,5.333,4.737,3.454,7.443,9.478,4.264,3.957,5.680
1,5.252,3.174,7.918,6.263,8.885,7.760,1.293,7.543,3.474,10.523,...,4.824,4.995,5.694,3.120,3.259,7.386,7.060,5.537,5.674,6.041
2,9.668,7.590,5.945,10.165,12.832,11.707,5.356,11.959,7.890,14.470,...,8.726,8.897,9.596,7.288,7.675,10.561,5.087,9.921,10.090,9.943
3,8.040,8.589,9.389,3.960,4.656,5.052,9.822,9.005,7.444,6.931,...,5.981,6.018,4.468,7.402,7.228,6.150,9.388,6.293,8.410,4.260
4,4.770,6.411,9.020,4.525,2.827,2.904,8.086,4.556,5.258,2.804,...,4.050,4.083,4.047,5.666,5.145,6.539,8.574,2.838,4.007,4.394


In [276]:
model = pyo.ConcreteModel('Mobian')

#Set index initialize
model.J = pyo.Set(initialize = junctions.index) #junctions
model.H = pyo.Set(initialize = hubs.index) #hubs
model.P = pyo.Set(initialize = pois.index)

#Parameters
# single parameters
model.Delta = pyo.Param(mutable = False, default = parameters['Delta'])
model.tau   = pyo.Param(mutable = False, default = parameters['tau'])
model.T     = pyo.Param(mutable = False, default = parameters['T'])
model.D     = pyo.Param(mutable = False, default = parameters['D'])
model.U     = pyo.Param(mutable = False, default = parameters['U'])

#distance from a location to another
model.d_jp = pyo.Param(model.J, model.P, 
                       initialize = lambda model, j, p: distances_jp.loc[j, p])

model.d_jh = pyo.Param(model.J, model.H, 
                       initialize = lambda model, j, h: distances_jh.loc[j, h])

model.d_hp = pyo.Param(model.H, model.P, 
                       initialize = lambda model, h, p: distances_hp.loc[h, p])

#car/bike time from a location to another
model.t_jp = pyo.Param(model.J, model.P, 
                       initialize = lambda model, j, p: time_jp.loc[j, p])

model.t_jh = pyo.Param(model.J, model.H, 
                       initialize = lambda model, j, h: time_jh.loc[j, h])

model.t_hp = pyo.Param(model.H, model.P,
                       initialize = lambda model, h, p: time_hp.loc[h, p])

#demand from a junction to a poi
model.n_jp = pyo.Param(model.J, model.P,
                       initialize = lambda model, j, p: demand.loc[j, p])
#variables
model.x = pyo.Var(model.H, within = pyo.Binary)
model.z = pyo.Var(model.J, model.P, within = pyo.Binary)

In [283]:
# Constraint on delta
@model.Constraint(model.J, model.P)
def maximum_extra_time(model, j, p):

    temp = pyo.quicksum(model.x[h] for h in model.H 
                       if model.t_jh[j,h]+model.t_hp[h,p]-model.t_jp[j,p] <= model.Delta)
    
    return model.z[j, p] <= temp



'pyomo.core.base.constraint.ScalarConstraint'>) on block Mobian with a new
Component (type=<class
'pyomo.core.base.constraint.AbstractScalarConstraint'>). This is usually
indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().
ERROR: Rule failed when generating expression for Constraint extra_time with
index None: ValueError: Constraint 'extra_time' does not have a proper value.
Found 'False' Expecting a tuple or relational expression. Examples:
       sum(model.costs) == model.income (0, model.price[item], 50)
ERROR: Constructing component 'extra_time' from data=None failed: ValueError:
Constraint 'extra_time' does not have a proper value. Found 'False' Expecting
a tuple or relational expression. Examples:
       sum(model.costs) == model.income (0, model.price[item], 50)


ValueError: Constraint 'extra_time' does not have a proper value. Found 'False'
Expecting a tuple or relational expression. Examples:
   sum(model.costs) == model.income
   (0, model.price[item], 50)

In [ ]:
model.t_jp['A4', 'Oostzanerwerf Amsterdam']

IndexError: invalid index to scalar variable.

In [ ]:
dup = distances_jp.T.duplicated()
for i in range(len(dup)):
    if dup[i]:
        print(dup.keys()[i])

Sloterdijk Amsterdam
1069 PJ Amsterdam
1083 Amsterdam


In [ ]:
distances_jp.T.drop_duplicates().T['Sloterdijk Amsterdam']

Roads
A4    5.102
A5    2.098
A8    6.161
A1    8.836
A2    7.100
Name: Sloterdijk Amsterdam, dtype: float64

In [ ]:
#5.102,2.098,6.161,8.836,7.100
data['d_jh']#.set_index(list(junctions.index))


,1,2,3,4,5,6,7,8,9,10,...,147,148,149,150,151,152,153,154,155,156
0,,,,,,,,,,,,,,,,,,,,,
3.930,3.375,9.924,5.902,7.612,6.487,5.661,5.815,2.997,9.250,5.056,...,4.138,4.255,5.333,4.737,3.454,7.443,9.478,4.264,3.957,5.680
5.252,3.174,7.918,6.263,8.885,7.760,1.293,7.543,3.474,10.523,6.379,...,4.824,4.995,5.694,3.120,3.259,7.386,7.060,5.537,5.674,6.041
9.668,7.590,5.945,10.165,12.832,11.707,5.356,11.959,7.890,14.470,10.795,...,8.726,8.897,9.596,7.288,7.675,10.561,5.087,9.921,10.090,9.943
8.040,8.589,9.389,3.960,4.656,5.052,9.822,9.005,7.444,6.931,8.115,...,5.981,6.018,4.468,7.402,7.228,6.150,9.388,6.293,8.410,4.260
4.770,6.411,9.020,4.525,2.827,2.904,8.086,4.556,5.258,2.804,3.666,...,4.050,4.083,4.047,5.666,5.145,6.539,8.574,2.838,4.007,4.394
